In [1]:
using StatisticalRethinking, CmdStan
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
first(df2, 5)

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real<lower=0> h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=0,upper=250> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchains);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.1s.jls", chn)
chn2 = deserialize("m4.1s.jls")

describe(chn2)


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.033, 0.033, 0.033, 0.033) seconds, 0.13 seconds total
Sampling took (0.055, 0.049, 0.048, 0.045) seconds, 0.20 seconds total

                Mean     MCSE  StdDev    5%   50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -892  2.7e-02     1.1  -894  -891  -891  1.8e+03  8.9e+03  1.0e+00
accept_stat__   0.91  1.5e-03    0.11  0.69  0.95   1.0  5.0e+03  2.5e+04  1.0e+00
stepsize__      0.84  1.1e-02   0.016  0.82  0.85  0.86  2.0e+00  1.0e+01  1.2e+13
treedepth__      1.9  1.1e-02    0.60   1.0   2.0   3.0  3.0e+03  1.5e+04  1.0e+00
n_leapfrog__     4.4  7.4e-02     3.8   1.0   3.0    11  2.7e+03  1.4e+04  1.0e+00
divergent__     0.00      nan    0.00  0.00  0.00  0.00      nan      nan      nan
energy__         893  3.9e-02     1.5   891   892   896  1.6e+03  7.9e+03  1.0e+00
sigma            7.8  5.6e-03    0.31   7.3   7.

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean    │ std      │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64 │ Float64  │ Float64    │ Float64    │ Any    │
├─────┼────────────┼─────────┼──────────┼────────────┼────────────┼────────┤
│ 1   │ mu         │ 154.609 │ 0.432361 │ 0.00683622 │ 0.00673822 │ 4000.0 │
│ 2   │ sigma      │ 7.77284 │ 0.307828 │ 0.00486719 │ 0.00544078 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%    │ 25.0%   │ 50.0%   │ 75.0%   │ 97.5%   │
│     │ Symbol     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ mu         │ 153.768 │ 154.317 │ 154.608 │ 154.897 │ 155.47  │
│ 2   │ sigma      │ 7.1841  │ 7.56068 │ 7.76757 │ 7.97718 │ 8.38197 │


end of m4.1s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*